In [1]:
date = 20220930

import numpy as np
import pandas as pd
import pandasql
import sqlite3

#pd.set_option("display.max_columns", None) 
#pd.set_option("display.max_colwidth", 1000) #huruf dlm column
#pd.set_option("display.max_rows", 1000)
#pd.set_option("display.precision", 2) #2 titik perpuluhan

#SINI
LCR =  pd.read_excel(r"T:\MIB Risk Management\ERM\18.0_LCR\Source_Data\\"+str(date)+"\\Gap Liquidity_"+str(date)+".xlsx", index = False)

Product_Hierarchy = pd.read_csv (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\06_Product_Hierarchy\Product_Hierarchy.txt", sep = "\t", header = 0, low_memory = False)
Product_Hierarchy.columns = Product_Hierarchy.columns.str.replace(" ", "_")

Al_Mapping =  pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\99_Adhoc\Syahid\A\User defined\Al_Mapping.xlsx", index = False)


hrc = Product_Hierarchy[['Source_Prod_Type_Dsc','Level_4','Level_6','Product_Hierarchy_Cd']]

L000 = pd.merge(LCR, hrc.rename(columns={'Product_Hierarchy_Cd': 'CONTRACT_TYPE'}),\
                on = 'CONTRACT_TYPE', how = 'left')
L000 = L000.fillna(0)

L001 = L000.iloc[np.where((L000.ATTRIBUTE_1=="N")&(L000.TB_ID != 0))]

def Bucket_Group(tb_id):
    if (tb_id >= 1) & (tb_id <= 7): 
        return "01.UP TO 1 WEEK"
    elif (tb_id > 7) & (tb_id <= 30): 
        return "02.>1 WEEK - 1 MTH"
    elif (tb_id > 30) & (tb_id <= 61): 
        return "03.>1 MTH - 3 MTHS"
    elif (tb_id > 61) & (tb_id <= 64): 
        return "04.>3 MTHS - 6 MTHS"
    elif (tb_id > 64) & (tb_id <= 70): 
        return "05.>6 MTHS - 1 YR"
    elif (tb_id > 70):
        return "06.>1 YR"
#    else:
#        return "07.NO SPECIFIC MATURITY"
L001['Bucket_Group'] = L001['TB_ID'].apply(Bucket_Group)

def CATEGORY_NEW(attribute_3):
    if "_" in attribute_3:
        return attribute_3[:-2]
    else:
        return attribute_3
L001['CATEGORY_NEW'] = L001['ATTRIBUTE_3'].apply(CATEGORY_NEW)

L001['LCR_BAL'] = L001['SUM_N_FLOW_GAP'] + L001['SUM_I_FLOW_GAP']

L002 = pd.merge(L001, Al_Mapping.rename(columns={'category': 'CATEGORY_NEW'}),\
                on = 'CATEGORY_NEW',how = 'left')
L002 = L002.fillna(0)

CASHFLOW_SUMM = L002.fillna(0).groupby(['#PARTITION_KEY',
'ATTRIBUTE_9',
'assets_liabilities',
'ATTRIBUTE_3',
'CONTRACT_TYPE',
'Level_4',
'Level_6',
'Bucket_Group',
'TB_ID'])[['LCR_BAL']].sum().reset_index().rename(columns={'TB_ID':'ROW_ID','ATTRIBUTE_9':'MARKET_SEGMENT','ATTRIBUTE_3':'CATEGORY'})

CASHFLOW_SUMM = CASHFLOW_SUMM.fillna(0)

def PRODUCT_GROUP_IA(Level_4,Level_6,MARKET_SEGMENT,contract_type):
    #NON-RETAIL FINANCING
    if( ((Level_4 != "Auto")&(Level_4 != "Cards"))&\
       ((Level_4 == "Term Loans")|(Level_4 == "OD")|(Level_4 == "Trade")|(Level_4 == "Short Term Revolving Credit"))&\
       ((Level_6 != "Salary")&(Level_6 != "SMF")) ):
        if ((MARKET_SEGMENT == "5GWB-CORP GLC")|(MARKET_SEGMENT == "6GWB-CORP MC")|(MARKET_SEGMENT == "7GWB-CORP MNC")|\
            (MARKET_SEGMENT == "8GWB-INTERNATIONAL")):
            return '01. Corporate Banking'
        elif (MARKET_SEGMENT == "2CFS-BB"):
            return '02. Business Banking'
        elif (MARKET_SEGMENT == "1CFS-SME"):
            return '03. RSME Banking'
    #RETAIL FINANCING
    #MORTGAGE FINANCING
    if((Level_4 == "Term Loans")&((MARKET_SEGMENT =="3CFS-IND")|(MARKET_SEGMENT =="4CFS-OTHER ENTITIES")|(MARKET_SEGMENT == 0)) ):
        if (Level_6 == "Housing"):
            return '04. Home Financing'
        elif (Level_6 == "Shophouse"):
            return '05. Shophouse'
        elif (Level_6 == "Other Mortgage Loan"):
            return '06. Other Mortgage'
    #HIRE PURCHASE
    if (Level_4 == "Auto"):
        return '07. Hire Purchase'
    #CREDIT CARDS
    if ((Level_4 == "Cards")|(Level_4 == "Term Loans")&((contract_type == '711')|(contract_type == '714'))):
        return '08. Credit Cards'
    #UNIT TRUST
    if ((Level_4 == "Term Loans")&((MARKET_SEGMENT =="3CFS-IND")|(MARKET_SEGMENT =="4CFS-OTHER ENTITIES")|(MARKET_SEGMENT == 0)) ):
        if (Level_6 == 'Unit Trust'):
            return '09. Unit Trust'
    #SALARY
    if (Level_6 == 'Salary'):
        return '11. Salary'
    #OTHER RETAIL FINANCING
    if ((Level_4 == "Term Loans")&((MARKET_SEGMENT =="3CFS-IND")|(MARKET_SEGMENT =="4CFS-OTHER ENTITIES")|(MARKET_SEGMENT == 0)) ):
        if ( ((Level_6 != "Housing")&(Level_6 != "Shophouse")&(Level_6 != "Other Mortgage Loan")&\
              ((Level_6 != 'Salary'))&(Level_6 != 'Unit Trust'))&(Level_4 == "Term Loans")&((contract_type != '711')&(contract_type != '714')) ):
            return '12. Other Personal Financing'
    #CASHLINE
    if ((Level_4 == "OD")&((MARKET_SEGMENT =="3CFS-IND")|(MARKET_SEGMENT =="4CFS-OTHER ENTITIES")|(MARKET_SEGMENT == 0))&(Level_6 != 'SMF') ):
        return '13. Cashline'
    #SMF
    if (Level_6 == 'SMF'):
        return '13. SMF'
    #MANUAL ADJUSTMENT
    if (Level_4 == 0):
        if (MARKET_SEGMENT == '1CFS-SME'):
            return '03. RSME Banking'
        elif (MARKET_SEGMENT == '2CFS-BB'):
            return '02. Business Banking'
        elif (((MARKET_SEGMENT == '5GWB-CORP GLC')|(MARKET_SEGMENT == '6GWB-CORP MC')|(MARKET_SEGMENT == '7GWB-CORP MNC')|(MARKET_SEGMENT == '8GWB-INTERNATIONAL'))&((contract_type == 'ABI_OWN')|(contract_type == 'MTR_I'))):
            return '01. Corporate Banking'
CASHFLOW_SUMM['PRODUCT_GROUP_IA'] = CASHFLOW_SUMM.apply(lambda x: PRODUCT_GROUP_IA(x['Level_4'], x['Level_6'],x['MARKET_SEGMENT'],x['CONTRACT_TYPE']), axis=1)

#==============================================================================

CASHFLOW_SUMM["PRODUCT_GROUP_IA"].fillna(0, inplace = True)

CASHFLOW_SUMM.loc[((CASHFLOW_SUMM.Level_4 == 0)& (((CASHFLOW_SUMM.CONTRACT_TYPE == 'ABI_OWN')|(CASHFLOW_SUMM.CONTRACT_TYPE == 'MTR_I') ) & (CASHFLOW_SUMM.PRODUCT_GROUP_IA != 0))), 'Level_4'] = 'Trade'
CASHFLOW_SUMM.loc[((CASHFLOW_SUMM.Level_4 == 0)& ~(((CASHFLOW_SUMM.CONTRACT_TYPE == 'ABI_OWN')|(CASHFLOW_SUMM.CONTRACT_TYPE == 'MTR_I') ) & (CASHFLOW_SUMM.PRODUCT_GROUP_IA != 0))), 'Level_4'] = 'OD'


#data LCR.CASHFLOW_SUMM3_&RDATE;
#set LCR.CASHFLOW_SUMM_&RDATE;
#if PRODUCT_GROUP_IA not in ('08. Credit Cards','') and UPCASE(LEVEL_4) not in ('OD','TRADE');
#run;

CASHFLOW_SUMM3 = CASHFLOW_SUMM[~(CASHFLOW_SUMM.PRODUCT_GROUP_IA.isin(['08. Credit Cards']))\
                               & ~(CASHFLOW_SUMM['Level_4'].isin(['Trade', 'OD']))&(CASHFLOW_SUMM.PRODUCT_GROUP_IA != 0)]


#SINI
writer2 = pd.ExcelWriter(r"T:\MIB Risk Management\ERM\18.0_LCR\Source_Data\\"+str(date)+"\\CASHFLOW_SUMM_"+str(date)+"(LCR python).xlsx",engine='xlsxwriter')

CASHFLOW_SUMM.to_excel(writer2, sheet_name='CASHFLOW', index = False)
CASHFLOW_SUMM3.to_excel(writer2, sheet_name='CASHFLOW3', index = False)

writer2.save()

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

# DB SECTION
SQLite in Python (Outsource)

In [ ]:
c.execute("drop table L002")
conn.commit() 

In [ ]:
cek = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name ;", conn)
cek

In [ ]:
conn.close()

In [ ]:
#sample coding using sqlite3

conn = sqlite3.connect("LCR.db")
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS L002 ({})".format(','.join(L002.columns)))

for row in L002.iterrows():
    sql = "INSERT INTO L002({}) VALUES ({})".format(','.join(L002.columns), ','.join(['?']*len(L002.columns)))
    c.execute(sql, tuple(row[1]))

conn.commit()

#boleh terus from L002 just rename, sort n remove column

c.execute("""
CREATE TABLE CASHFLOW_SUMM AS SELECT
partition_key,
ATTRIBUTE_9 AS MARKET_SEGMENT,
assets_liabilities,
attribute_3 as CATEGORY,
CONTRACT_TYPE,
Level_4,
Level_6,
BUCKET_GROUP,
tb_id as ROW_ID,
SUM(LCR_BAL)AS LCR_BAL
FROM L002
GROUP BY 1,2,3,4,5,6,7,8,9
; """)

conn.commit()

CASHFLOW_SUMM = pd.read_sql_query("SELECT * FROM CASHFLOW_SUMM", conn)

#CASHFLOW_SUMM["MARKET_SEGMENT"].fillna("0", inplace = True)
#CASHFLOW_SUMM["Level_4"].fillna(0, inplace = True)